In [3]:
# prompt: pip install catboost

%pip install catboost


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:


import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
import lightgbm as lgb
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error
#from catboost import CatBoostClassifier, Pool, cv
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb


In [5]:

train_set = pd.read_csv(r'C:\Users\hrahe\Downloads\train_set.csv')
test_set = pd.read_csv(r'C:\Users\hrahe\Downloads\test_set.csv')

In [6]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [7]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [9]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [25]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split

# # ... (your existing code) ...

# # Convert X and X_testdata back to DataFrames if they are NumPy arrays
# if isinstance(X, np.ndarray):
#     X = pd.DataFrame(X, columns=train_set.drop(columns=['Y', 'RecordId']).columns)  # Use original column names
# if isinstance(X_testdata, np.ndarray):
#     X_testdata = pd.DataFrame(X_testdata, columns=test_set.drop(columns=['RecordId']).columns)  # Use original column names


# # Calculate correlations with the target variable
# correlations = X.corrwith(y)

# # Select features with correlation value greater than 0.01
# selected_features = correlations[correlations.abs() > 0.02].index

# # Print the selected features
# print("Selected Features (Correlation > 0.01):")
# for feature in selected_features:
#     print(f"{feature} (Correlation: {correlations[feature]:.3f})")

# # Filter X and X_testdata based on selected features
# X = X[selected_features]
# X_testdata = X_testdata[selected_features]

# # ... (rest of your code) ...

In [10]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [11]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Univariate feature selection
selector = SelectKBest(score_func=mutual_info_classif, k=55)

# Fit and transform the training data
x_train_reduced = selector.fit_transform(X_train, y_train)

# Transform the test data
x_test_reduced = selector.transform(X_test)
# df_test_reduced = selector.transform(df_test_standard)
X_testdata=selector.transform(X_testdata)
# Now you can proceed with training your model using x_train_reduced and y_train
# and evaluate it using x_test_reduced and df_test_reduced

In [12]:
X_test=x_test_reduced
X_train=x_train_reduced

In [29]:
scalar=StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test=scalar.transform(X_test)
X_testdata = scalar.transform(X_testdata)




In [13]:
# prompt: make stacking classifier with lgm catboost and lg as final estimator

# Assuming you have already defined X_train, y_train, X_test, y_test

# Initialize the individual models with their best parameters (replace with your actual best parameters)
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, classification_report
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(
    objective='binary',
    metric='binary_logloss',
    boosting_type='gbdt',
    num_leaves=20,
    max_depth=4,
    learning_rate=0.01,
    n_estimators=1212,
    
    scale_pos_weight=1.1,
    reg_lambda=0.1,
    verbose=-1
)

catboost_model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    iterations=2000,
    learning_rate=0.01,
    depth=4,
  
    task_type='CPU'  # Use GPU if available
)
lgb_model=lgb.LGBMClassifier(
        objective='binary',
        metric='binary_logloss',
        boosting_type='gbdt',
        num_leaves=20,
        max_depth=4,
        learning_rate=0.01,
        n_estimators=1222,
        subsample=0.7,
       
        verbose=-1
    )
# Create a stacking classifier
stacking_clf = StackingClassifier(
    estimators=[
        ('lgb', lgb_model),
        ('catboost', catboost_model)
    ],
    final_estimator=lgb_model,
    stack_method='predict_proba'
)

# Train the stacking classifier
stacking_clf.fit(X_train, y_train)

# Make predictions
y_pred = stacking_clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# You can also calculate other metrics like AUC, precision, recall, etc. based on your needs.

0:	total: 208ms	remaining: 6m 55s
1:	total: 238ms	remaining: 3m 57s
2:	total: 268ms	remaining: 2m 58s
3:	total: 293ms	remaining: 2m 26s
4:	total: 319ms	remaining: 2m 7s
5:	total: 347ms	remaining: 1m 55s
6:	total: 371ms	remaining: 1m 45s
7:	total: 395ms	remaining: 1m 38s
8:	total: 418ms	remaining: 1m 32s
9:	total: 441ms	remaining: 1m 27s
10:	total: 464ms	remaining: 1m 23s
11:	total: 487ms	remaining: 1m 20s
12:	total: 513ms	remaining: 1m 18s
13:	total: 536ms	remaining: 1m 15s
14:	total: 561ms	remaining: 1m 14s
15:	total: 584ms	remaining: 1m 12s
16:	total: 606ms	remaining: 1m 10s
17:	total: 630ms	remaining: 1m 9s
18:	total: 656ms	remaining: 1m 8s
19:	total: 683ms	remaining: 1m 7s
20:	total: 708ms	remaining: 1m 6s
21:	total: 734ms	remaining: 1m 6s
22:	total: 760ms	remaining: 1m 5s
23:	total: 787ms	remaining: 1m 4s
24:	total: 815ms	remaining: 1m 4s
25:	total: 842ms	remaining: 1m 3s
26:	total: 869ms	remaining: 1m 3s
27:	total: 899ms	remaining: 1m 3s
28:	total: 930ms	remaining: 1m 3s
29:	tota

c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [14]:
y_pred = stacking_clf.predict(X_test)
y_pred_proba = stacking_clf.predict_proba(X_test)[:, 1] # Get probabilities for the positive class


# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred_proba)) # Calculate and print AUC

Accuracy: 0.9976759682111456
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     61388
           1       0.00      0.00      0.00       143

    accuracy                           1.00     61531
   macro avg       0.50      0.50      0.50     61531
weighted avg       1.00      1.00      1.00     61531

AUC: 0.9484647349132265


c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [15]:
# Predict probabilities for the test set
y_pred_proba_testdata = stacking_clf.predict_proba(X_testdata)[:, 1]

# Create a DataFrame with RecordId and predicted probabilities
test_set['Y_probability'] = y_pred_proba_testdata

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities.csv', index=False)

print("Probabilities saved to test_set_with_probabilities.csv")

import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)

Probabilities saved to test_set_with_probabilities.csv
[5.94600630e-05 2.11165909e-03 1.81523980e-06 ... 3.62416934e-02
 2.34788946e-03 1.31949851e-03]
        RecordId  Y_probability
0         300001       0.000059
1         300002       0.002112
2         300003       0.000002
3         300004       0.000138
4         300005       0.000004
...          ...            ...
105477    405478       0.000002
105478    405479       0.246454
105479    405480       0.000057
105480    405481       0.000006
105481    405482       0.000008

[105482 rows x 2 columns]
